In [2]:
#Step 1. 필요한 모듈과 라이브러리를 import 합니다

import pandas as pd 
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import Select
from selenium.webdriver.common.action_chains import ActionChains
import time
import sys
import os
import math
import numpy
import xlwt

keyword = input('1.크롤링할 키워드는 무엇입니까?(예:여행): ')
in_ = list(input('2.포함할 키워드를 입력하세요.(예:환전,면세):').split(','))
ex_ = list(input('3.제외할 키워드를 입력하세요.(예:중국,미국):').split(','))
start_d = input('4.조회를 시작할 날짜를 입력하세요.(예:20200101):')
end_d = input('5.조회를 종료할 날짜를 입력하세요.(예:20201231):')
how_many = int(input('6.크롤링할 건수는 몇건입니까?:'))
f_dir = input('7.결과 파일을 저장할 폴더명을 쓰세요(예:c:\\data\\):')
n = time.localtime()
s = '%04d-%02d-%02d-%02d-%02d-%02d' % (n.tm_year, n.tm_mon, n.tm_mday, n.tm_hour, n.tm_min, n.tm_sec)
os.makedirs(f_dir+s+'-'+keyword)
os.chdir(f_dir+s+'-'+keyword)
f_txt=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.txt'
f_csv=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.csv'
f_xls=f_dir+s+'-'+keyword+'\\'+s+'-'+keyword+'.xlsx'
path = "C:\Temp\chromedriver_win32\chromedriver.exe"
driver = webdriver.Chrome(path)
driver.get("https://www.naver.com")
time.sleep(1)

url = "https://search.naver.com/search.naver?where=nexearch&sm=top_hty&fbm=0&ie=utf8&query=" + keyword
url = url + '+%2B' + '+%2B'.join(in_) + '+-' + '+-'.join(ex_) + "&sm=tab_opt&nso=p%3Afrom" + start_d + "to" + end_d
driver.get(url)
time.sleep(1)
driver.find_element(By.LINK_TEXT,"VIEW").click()
time.sleep(1)
driver.find_element(By.LINK_TEXT,"블로그").click()

for _ in range(int(how_many / 30)):
    body = driver.find_element(By.CSS_SELECTOR, 'body')
    body.send_keys(Keys.END)
    time.sleep(3)
n=1
n2 = 1
nl = []
urls = []
titles = []
nicks = []
dates = []
cons = []

soup = BeautifulSoup(driver.page_source, 'html.parser')
content_list = soup.find('ul', class_='lst_total')

for i in content_list.find_all('li', "bx"):
    if n <= how_many:
        url = i.find('a', 'api_txt_lines total_tit').get_text()
        url = i.find('a', 'api_txt_lines total_tit').get("href")
        urls.append(url)
        n += 1
    else:
        break
        
for i in urls:
    driver.get(i)
    time.sleep(1)
    try:
        driver.switch_to.frame('mainFrame')
    except:
        print("")
    soup = BeautifulSoup(driver.page_source, 'html.parser')
    nl.append(n2)
    n2 += 1
    title = soup.find('div', 'pcol1')
    if title is None:
        title = soup.find('div', 'htitle')
    if title is None:
        title = soup.find('div', 'titleWrap jumbotron')
    try:
        title = title.get_text().replace("\n","")
    except:
        title = "추출실패"
    titles.append(title)

    nick = soup.find('span', 'nick')
    if nick is None:
        nick = soup.find('div', 'nick')
    try:
        nick = nick.get_text()
    except:
        nick = "추출실패"
    nicks.append(nick)

    date = soup.find('span', 'se_publishDate pcol2')
    if date is None:
        date = soup.find('p', 'date fil5 pcol2 _postAddDate')
    try:
        date = date.get_text()
    except:
        date = "추출실패"
    dates.append(date)

    con = soup.find('div', 'se-main-container')
    if con is None:
        con = soup.find('div', id='postViewArea')
    if con is None:
        con = soup.find('div', 'se_component_wrap sect_dsc __se_component_area')
    try:
        con = con.get_text().replace("\n","")
    except:
        con = "추출실패"
    cons.append(con)
    
df = pd.DataFrame()
df['제목'] = titles
df['주소'] = urls
df['작성자'] = nicks
df['작성일'] = dates
df['내용'] = cons
df.to_excel(f_xls)
df.to_csv(f_csv, encoding="utf-8-sig")       
f = open(f_txt, 'a',encoding='UTF-8')
for i in range(len(nl)):
    f.write('총 '+ str(how_many) + '건중 ' + str(nl[i]) + '번째 블로그 데이터를 수집합니다. ~~\n')
    f.write('1.제목: ' + titles[i] + '\n')
    f.write('2.주소: ' + urls[i] + '\n')
    f.write('3.작성자: ' + nicks[i] + '\n')
    f.write('4.작성일: ' + dates[i] + '\n')
    f.write('5.내용: ' + cons[i] + '\n\n')
f.close()
print("요청하신 데이터 추출이 성공적으로 끝났습니다.")

1.크롤링할 키워드는 무엇입니까?(예:여행): 여행
2.포함할 키워드를 입력하세요.(예:환전,면세):환전
3.제외할 키워드를 입력하세요.(예:중국,미국):중국
4.조회를 시작할 날짜를 입력하세요.(예:20200101):20200101
5.조회를 종료할 날짜를 입력하세요.(예:20201231):20201231
6.크롤링할 건수는 몇건입니까?:28
7.결과 파일을 저장할 폴더명을 쓰세요(예:c:\data\):C:\Users\UIT801-\Desktop\20191469\3학년 1학기\SNS빅데이터\과제\SNS_기말과제\Q2\


C:\Users\UIT801-\AppData\Local\Temp\ipykernel_17116\3906059191.py:32: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(path)






요청하신 데이터 추출이 성공적으로 끝났습니다.
